1. Data loading and initial exploration

Load the dataset into a pandas DataFrame
Explore the first few rows to understand the structure of data
Check the data types, summary statistics, and unique values of each column
Identify any data quality issues or inconsistencies

In [15]:
import pandas as pd

df=pd.read_csv("../CaseStudy3/SuperStore_Dataset.csv")
# Display the first 10 rows of the DataFrame    
first_10_rows_data = df.head(10)
print(f"First 10 rows of data:\n {first_10_rows_data} \n")
# Display the last 10 rows of the DataFrame    
last_10_rows_data = df.tail(10)
print(f"Last 10 rows of data: \n {last_10_rows_data}")


First 10 rows of data:
    Row ID        Order ID Order Date  Ship Date       Ship Mode Customer ID  \
0       1  CA-2016-152156   08/11/16   11/11/16    Second Class    CG-12520   
1       2  CA-2016-152156   08/11/16   11/11/16    Second Class    CG-12520   
2       3  CA-2016-138688   12/06/16   16/06/16    Second Class    DV-13045   
3       4  US-2015-108966   11/10/15   18/10/15  Standard Class    SO-20335   
4       5  US-2015-108966   11/10/15   18/10/15  Standard Class    SO-20335   
5       6  CA-2014-115812   09/06/14  2014-6-14  Standard Class    BH-11710   
6       7  CA-2014-115812   09/06/14  2014-6-14  Standard Class    BH-11710   
7       8  CA-2014-115812   09/06/14  2014-6-14  Standard Class    BH-11710   
8       9  CA-2014-115812   09/06/14  2014-6-14  Standard Class    BH-11710   
9      10  CA-2014-115812   09/06/14  2014-6-14  Standard Class    BH-11710   

     Customer Name    Segment        Country             City  ...  \
0      Claire Gute   Consumer  Unite